In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-09/data/holes_GAMA-09_ukidss_k_O16_20180417.fits')


## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_GAMA-09/data/GAMA9_Ldust_prediction_results.fits')


In [5]:
cigale['id'].name = 'help_id'


In [6]:
cigale

help_id,bayes.dust.luminosity,bayes.dust.luminosity_err,best.omegacam_u,best.megacam_u,best.omegacam_g,best.suprime_g,best.decam_g,best.megacam_g,best.gpc1_g,best.suprime_r,best.gpc1_r,best.megacam_r,best.omegacam_r,best.decam_r,best.gpc1_i,best.megacam_i,best.omegacam_i,best.suprime_i,best.gpc1_z,best.vista_z,best.megacam_z,best.suprime_z,best.decam_z,best.gpc1_y,best.suprime_y,best.vista_y,best.ukidss_y,best.ukidss_j,best.vista_j,best.ukidss_h,best.vista_h,best.vista_ks,best.ukidss_k
,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
str27,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J082912.113+014254.016,1.77568115836e+39,1.85609942134e+39,0.000684965564886,0.000821352521557,0.00144183207848,0.00148516352542,0.00149180973989,0.00152484207857,0.00152236399701,0.00292362440639,0.00293234518063,0.00299420445508,0.00307194580642,0.0032099465634,0.00570727344152,0.00587427379469,0.00612442347805,0.00651726375172,0.0120436680647,0.0123872369988,0.0127309395159,0.0128854969896,0.013894760725,0.0156775037919,0.0159951505172,0.0168155160421,0.0170408681251,0.0242142967252,0.0242938067077,0.0362444274603,0.0365352086652,0.053322189509,0.0544946185725
HELP_J082913.120+014309.267,2.70212657034e+37,2.4317979539e+36,0.0573110432967,0.0562235789619,0.0603379698001,0.0608231803715,0.0609571425584,0.0609625599188,0.0612316883795,0.0583285006846,0.0580161745538,0.0574112860669,0.0574133612291,0.0569884758346,0.0528267510041,0.0531286069587,0.0530603414379,0.0526726986733,0.0511368015529,0.0510961639642,0.0512045776455,0.0512039835976,0.0512222638582,0.0510054297275,0.0508941799581,0.0516693579822,0.0520356898121,0.0535853198261,0.0535460508147,0.0523495387546,0.0523398453257,0.0526552391408,0.051438963437
HELP_J082913.510+014205.882,4.57019486182e+37,2.88564478964e+37,0.00156277502458,0.00192418494454,0.00407346583998,0.00438945893704,0.00444533881342,0.00461453381606,0.00461774997902,0.0133383483064,0.0134180824496,0.0137568580713,0.0140218194494,0.0145558588316,0.0210167471998,0.0209496858541,0.0212616611587,0.0218665518448,0.0274352226069,0.0279383457913,0.0284199463628,0.0287349611616,0.0300696478973,0.0326876826713,0.0336146737958,0.0365426207371,0.0371574409942,0.0523312884338,0.0525172784308,0.0693596956616,0.0697049461743,0.0912814885421,0.0938379817377
HELP_J082913.660+014249.800,1.5495572579e+39,2.04264123065e+39,0.000711345808592,0.000862099824131,0.00146368041862,0.00151836539645,0.00152890794083,0.00156506147189,0.00156550484369,0.00348666657406,0.00349250734196,0.00356912491012,0.00370763299483,0.00397475525761,0.00955537349445,0.00951497814732,0.00983445508732,0.0105356265145,0.0170907148285,0.0177239265424,0.0179956070582,0.0184064532071,0.0193826061142,0.0212997145429,0.0220023922129,0.0245431099141,0.0252827344276,0.0373282509981,0.0374210562827,0.061449745967,0.0621140795666,0.0896828507594,0.0920299893409
HELP_J082914.042+014215.484,1.98368311749e+39,1.12470536961e+38,6.29802453594,6.13842721723,6.47887404415,6.49912938738,6.50546878227,6.51727328203,6.52468948926,5.87729481567,5.84752162295,5.79201128916,5.78289937949,5.74194490447,5.25798553015,5.26936215959,5.26062085048,5.22271249463,5.05455001321,5.06117779025,5.070187866,5.08031109213,5.07033178201,5.02012905273,5.03477002704,5.15209101048,5.1711817136,5.15044948033,5.15409578925,5.05103468709,5.05493386165,4.91507578592,4.79307583793
HELP_J082914.054+014304.068,9.00848445891e+35,6.91995525496e+35,0.0722025158505,0.0888029010944,0.206479724459,0.210573006777,0.21132906029,0.217344030318,0.216248401557,0.34347803063,0.344869563689,0.349239902934,0.354105019616,0.362324863205,0.444102665295,0.444686001857,0.448793361436,0.455616499345,0.535951618509,0.546621011029,0.557597

## Read in photoz

In [7]:
photoz=Table.read('../../dmu24/dmu24_GAMA-09/data/master_catalogue_gama-09_20171206_photoz_20180213_r_optimised.fits')

In [8]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
str27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6
HELP_J084930.013-020417.068,132.375053567,-2.07140772343,30302,0.821,0.3029,1.3791,0.798,-99.0,-99.0,-99.0,-99.0,0.747461512522,-99.0,-99.0,-99.0,-99.0,
HELP_J085343.238-011847.076,133.430156467,-1.31307660643,30303,1.1554,0.3875,1.9962,0.797,-99.0,-99.0,-99.0,-99.0,1.01768053487,-99.0,-99.0,-99.0,-99.0,
HELP_J084950.135-013553.799,132.458896567,-1.59827758443,30304,1.0387,0.3587,1.78,0.8,-99.0,-99.0,-99.0,-99.0,0.929027455624,-99.0,-99.0,-99.0,-99.0,
HELP_J090051.096-013904.046,135.212899567,-1.65112388143,30306,1.0914,0.3751,1.8732,0.796,-99.0,-99.0,-99.0,-99.0,0.969903446791,-99.0,-99.0,-99.0,-99.0,
HELP_J085606.834-010133.117,134.028473367,-1.02586584843,30307,1.0371,0.3628,1.7717,0.796,-99.0,-99.0,-99.0,-99.0,0.923257682576,-99.0,-99.0,-99.0,-99.0,
HELP_J090529.562-011041.306,136.373173467,-1.17814049643,30308,1.1091,0.3792,1.9078,0.797,-99.0,-99.0,-99.0,-99.0,0.981740596603,-99.0,-99.0,-99.0,-99.0,
HELP_J090046.918-011856.173,135.195490567,-1.31560348643,30312,0.9841,0.3466,1.6739,0.796,-99.0,-99.0,-99.0,-99.0,0.88334960273,-99.0,-99.0,-99.0,-99.0,
HELP_J085032.410-011540.279,132.635043567,-1.26118848343,30313,1.109,0.3792,1.9078,0.797,-99.0,-99.0,-99.0,-99.0,0.981740596603,-99.0,-99.0,-99.0,-99.0,
HELP_J085443.327-011651.769,133.680527667,-1.28104686043,30314,0.9931,0.3506,1.6899,0.796,-99.0,-99.0,-99.0,-99.0,0.888999651538,-99.0,-99.0,-99.0,-99.0,


## Join CIGALE and photoz tables

In [9]:
prior=join(cigale,photoz,keys='help_id')

In [10]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['z1_median']).to(u.cm))


In [11]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [12]:
prior['DEC'].name='Dec'

## Read in Maps

In [13]:

pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE250_v0.9.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE350_v0.9.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-09_SPIRE500_v0.9.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [14]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [15]:
## Set XID+ prior class

In [16]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'GAMA09_Ldust_prediction_results.fits',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [17]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [18]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 4959 tiles required for input catalogue and 102 large tiles
writing total_bytes=1588798331...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1588798331)... done.


SystemExit: 

In [ ]:
prior250.nsrc